<a href="https://colab.research.google.com/github/MaiSerry/DoctBot_grad/blob/main/main_code_DoctorBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 878.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5acf0538358ba13789c68cae1d28160b1c7c4669cc87263b41924b84aa2da31b
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification,AutoModelForSeq2SeqLM,pipeline
import torch
import whisper
import tensorflow as tf
from langdetect import detect, LangDetectException
import os

In [ ]:
SESSION_OUTPUT_LANGUAGE = "en"  # Default to English
SESSION_LANGUAGE_LOCKED = False # To ensure SESSION_OUTPUT_LANGUAGE is set only once


In [ ]:
model_name = "maiserry/DoctorBot_symptoms_disease"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

#translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ar-en")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import hf_hub_download
import joblib

path = hf_hub_download("maiserry/DoctorBot_symptoms_disease", "label_encoder.pkl")
label_encoder = joblib.load(open(path, 'rb'))


label_encoder.pkl:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

In [18]:
speech_model = whisper.load_model("large-v3")


100%|█████████████████████████████████████| 2.88G/2.88G [00:48<00:00, 64.1MiB/s]


In [63]:
import pandas as pd
## this data will used to map department & translation disease
df=pd.read_csv("/content/diseases_with_arabic.csv")
df.head()

,Disease,Department,arabic_disease
0,postoperative infection,General Surgery,العدوى بعد العملية الجراحية
1,cystitis,Urology,التهاب المثانة
2,urethral disorder,Urology,اضطراب مجرى البول
3,sciatica,Neurology,عرق النسا
4,spondylolisthesis,Orthopedics,الفقرة الفقارية


In [64]:
df.shape

(773, 3)

In [46]:
department_translations = {
    'Allergy': 'حساسيه',
    'Cardiology': 'أمراض القلب',
    'Colorectal Surgery': 'جراحة القولون والمستقيم',
    'Dentistry': 'طب الاسنان ',
    'Dermatology': 'امراض جلدية وتجميلية',
    'ENT': 'انف واذن وحنجره',
    'Emergency Medicine': 'Emergency Medicine',
    'Endocrinology': 'الطب الباطني / الغدد الصماء',
    'Gastroenterology': 'الجهاز الهضمي',
    'General Surgery': 'جراحة عامة',
    'Gynecology': 'طب النساء',
    'Hematology': 'أمراض الدم',
    'Immunology': 'Immunology',
    'Infectious Disease': 'امراض معدية',
    'Nephrology': 'أمراض الكلى',
    'Neurology': 'أمراض الأعصاب',
    'Neurosurgery': 'جراحة الأعصاب',
    'Nutrition': 'تغذاية',
    'Obstetrics': 'طب التوليد',
    'Oncology': 'الاورام',
    'Ophthalmology': 'طب العيون',
    'Orthopedics': 'العظام',
    'Pediatrics': 'طب الأطفال',
    'Plastic Surgery': 'جراحة تجميلية',
    'Psychiatry': 'الطب النفسي',
    'Pulmonology': 'أمراض الصدر',
    'Rheumatology': 'الروماتيزم',
    'Thoracic Surgery': 'جراحة الصدر',
    'Trauma Surgery': 'Trauma Surgery',
    'Urology': 'المسالك البولية',
    'Vascular Surgery': 'جراحة الأوعية الدموية',
}


In [ ]:
#pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.2 MB/s eta 0:00:00


In [ ]:
#from rapidfuzz import fuzz,process


In [53]:
translation_model_name = "facebook/m2m100_418M"
translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
"""dialect_to_fusha = {
    "بطني بتوجعني": "أشعر بألم في البطن",
    "نفسي مسدودة": "لا أشعر بالجوع",
    "عندى سخونية": "أعاني من الحمى",
    "مش قادر أقف": "أشعر بالإرهاق",
    "رجلى بتوجعني": "أعاني من ألم في الساق",
    "نفسى غمه عليا": "أشعر بالغثيان",
    "نفسى مسدوده":"لا أشعر بالجوع",
    "زورى بيوجعنى ":"اشعر بألم فى الحلق",
    "سوخنه":"حمى",
    "سوخنيه":"حمى",
    "راسي بتوجعني": "أعاني من صداع",
     # هضمية
    "عندى حموضة": "أعاني من حرقة في المعدة",
    "بشعر بثقل في بطني": "أعاني من عسر هضم",
    "معدتى بتوجعني": "أشعر بألم في المعدة",
    "بشعر بوجع تحت الضلوع": "أعاني من ألم في الجانب الأيمن/الأيسر",

    # تنفسية
    "نفسى بيقطع": "أعاني من ضيق تنفس",
    "مش قادر اخد نفس": "أعاني من صعوبة في التنفس",
    "بلغم": "لدي إفرازات مخاطية",

    # أمراض شائعة
    "عندى ضغط": "أعاني من ارتفاع ضغط الدم",
    "عندى سكر": "أعاني من مرض السكري",
    "عندى حساسية": "لدي رد فعل تحسسي",
    "عندى ربو": "أعاني من الربو",
    "عندى أنيميا": "أعاني من فقر الدم",

}"""

In [ ]:
# 3. text into phrases
#def split_to_phrases(text):
 #   return re.split(r"[،,؛]| و |\.|\n", text)

In [ ]:
"""from rapidfuzz import fuzz, process

def convert_to_fusha(text):
    threshold=80
    phrases = split_to_phrases(text)
    converted_phrases = []
    for phrase in phrases:
        match = process.extractOne(phrase, dialect_to_fusha.keys(), scorer=fuzz.ratio)
        if match:
            best_match, score, _ = match
            if score >= threshold:
                converted_phrases.append(dialect_to_fusha[best_match])
            else:
                converted_phrases.append(phrase)
        else:
            converted_phrases.append(phrase)
    return ". ".join(converted_phrases)
"""

In [81]:
## we canceled use this function and alternatively uesd df['arabic_disease'] to reduce the Response time
## but may use in transLate later like arabic chat
def translate_with_model(text, target_lang, source_lang=None):
    global translation_tokenizer, translation_model
    if not text or not text.strip():
        return ""
    if source_lang:
        translation_tokenizer.src_lang = source_lang
    forced_bos_id = translation_tokenizer.get_lang_id(target_lang)
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated_tokens = translation_model.generate(**inputs, forced_bos_token_id=forced_bos_id)
    translated_text = translation_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0] #
    return translated_text

In [55]:
translate_with_model("i have alot of pain", "ar", "en")

'أنا أعاني من الألم'

In [ ]:
# 6. final function to arabic
#def translate_dialect_to_english(text):
 #   phrases = split_to_phrases(text)
    #fusha_phrases = [convert_to_fusha(p) for p in phrases if p.strip()]
    #full_sentence = ". ".join(fusha_phrases)
   # full_sentence = convert_to_fusha(text)
    #print("🔍 الجملة بعد التحويل إلى الفصحى:", full_sentence)
    #return translate_with_model(full_sentence)


In [84]:
from openai import OpenAI
def translate_arabic_to_medical_english(arabic_text):

    client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key="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",
    )

    prompt_message =  f"""
    Translate the following Egyptian colloquial Arabic medical symptom description into concise, standard medical English suitable for a clinical setting.

     Only return the translated symptoms on **one line**, separated by commas.

    ❗ Do NOT include anything else — no headings, no explanations, no phrases like "Here is the translation" or "Translation:", just the symptom list. Arabic: {arabic_text} English:"""

    completion = client.chat.completions.create(
      model="deepseek/deepseek-r1-0528:free",
      messages= [
                {"role": "system", "content": "You are an expert medical translator specializing in Egyptian colloquial Arabic to medical English."},
                {"role": "user", "content": prompt_message}
            ],
    )

    return completion.choices[0].message.content.strip()

In [86]:
translate_arabic_to_medical_english('زورى وجعانى وعندى احتقان')

'throat pain, congestion'

In [102]:
translate_arabic_to_medical_english('بطنى وجعانى وحسه بصداع ')


'abdominal pain, headache'

In [70]:
import torch.nn.functional as F

def predict_disease(symptoms_in_english, top_n=3,threshold=0.30):

    inputs = tokenizer(symptoms_in_english, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    CONFIDENCE_THRESHOLD = threshold

    probabilities = F.softmax(outputs.logits, dim=1)[0]

    top_probs, top_indices = torch.topk(probabilities, k=top_n)


    if top_probs[0].item() < CONFIDENCE_THRESHOLD:
        return [("Insufficient Symptoms", 0.0, "General Practitioner")]

    predictions = []
    for prob, idx in zip(top_probs, top_indices):
        disease_index = idx.item()
        confidence = prob.item()


        try:
            predicted_disease = label_encoder.inverse_transform([disease_index])[0]
        except:
            predicted_disease = "Unknown"

        if predicted_disease in df['Disease'].values:
            department = df[df['Disease']==predicted_disease]['Department'].values[0]
        else:
            department =  "General Practitioner"

        predictions.append((predicted_disease, confidence, department))

    return predictions


In [24]:
predicted_disease_en = label_encoder.inverse_transform([0])[0]
predicted_disease_en

'abdominal aortic aneurysm'

In [71]:
predict_disease("My skin is itchy and I have a red rash")

[('dyshidrosis', 0.4701314866542816, 'Dermatology'),
 ('fungal infection of the skin', 0.1351594477891922, 'Dermatology'),
 ('acariasis', 0.035483647137880325, 'Dermatology')]

In [26]:
predict_disease("I feel deep bone pain that gets worse at night and affects my walking. I also feel very tired and have lost some weight recently")

[('Insufficient Symptoms', 0.0, 'General Practitioner')]

In [72]:
predict_disease('I feel shortness of breath and chest pain when walking')

[('pericarditis', 0.524592399597168, 'Cardiology'),
 ('aortic valve disease', 0.11649712175130844, 'Cardiology'),
 ('ischemic heart disease', 0.06809378415346146, 'Cardiology')]

In [28]:
predict_disease('My throat is sore and I have trouble swallowing')

[('peritonsillar abscess', 0.3588472902774811, 'ENT'),
 ('mucositis', 0.18835285305976868, 'Dentistry'),
 ('abscess of the pharynx', 0.1404564082622528, 'ENT')]

In [ ]:
#model.config.id2label.values()

In [91]:
def get_symptoms_from_audio(file_path):

    print(get_ui_text("transcribing_audio"))

    if not os.path.exists(file_path):
        print(get_ui_text("audio_file_not_found", file_path=file_path))
        return "", "en" # Return empty symptom and default lang

    try:
        result = speech_model.transcribe(file_path)
        transcribed_text = result["text"].strip()
        detected_lang_of_speech = result.get("language", "en")
        print(get_ui_text("transcribed_text_label", lang=detected_lang_of_speech, text=transcribed_text))

        if not transcribed_text:
            print(get_ui_text("transcription_empty"))

            return "", detected_lang_of_speech

        symptoms_for_model = transcribed_text
        if detected_lang_of_speech == "ar":
            output=translate_arabic_to_medical_english(transcribed_text)
            symptoms_for_model = output
            print(get_ui_text("translated_to_en_label", text=symptoms_for_model))

        return symptoms_for_model, detected_lang_of_speech

    except Exception as e:
        print(get_ui_text("transcription_error_detailed", e=str(e)))

        return "", "en"


In [30]:
# -- Text translations --

ui_texts = {
    "en": {
        "welcome": "👋 Welcome to AI Medical Diagnosis System!",
        "input_method_prompt": "Would you like to enter symptoms as text or speech? (text/speech): ",
        "invalid_input_method": "Invalid input. Please type \"text\" or \"speech\".",
        "speech_file_prompt": "🎙 Please provide the path to your recorded audio file: ",
        "audio_transcription_error": "Could not transcribe audio, audio was empty, or an error occurred. Please try again or switch to text.",
        "no_audio_file": "No audio file path provided.",
        "text_symptoms_prompt": "🩺 Please describe your symptoms: ",
        "symptoms_required": "\nSymptoms are required to proceed. Please provide your symptoms.",
        "must_provide_symptoms": "You must provide symptoms to continue.",
        "insufficient_symptoms_msg": "\n⚠️ Your symptoms are not specific enough for a diagnosis.",
        "recommended_dept_insufficient": "🏥 Recommended Department: General Practitioner",
        "provide_more_details": "💬 Please provide more details (e.g., duration, pain location, severity).",
        "add_more_symptoms_prompt": "➕ Add more symptoms (or press Enter to stop): ",
        "diagnosis_results_header": "\n✅ Diagnosis Results:",
        "disease_label": "🩺 Disease: ",
        "confidence_label": " (confidence:{confidence:.2f})",
        "recommended_dept_label": "   🏥 Recommended Department: ",
        "audio_file_not_found": "Audio file not found at: {file_path}",
        "transcription_empty": "Transcription resulted in empty text.",
        "transcription_error_detailed": "Error during audio transcription or translation: {e}",
        "transcribing_audio": "🎤 Transcribing audio...",
        "transcribed_text_label": "🗣 Transcribed Text ({lang}): {text}",
        "translated_to_en_label": "🌍 Translated to English for model: {text}",
        "detected_text_lang_label": "Detected input language (text): {lang}",
        "lang_detection_error": "Could not detect language from text, assuming English for model. UI will use default.",
        "session_lang_set_debug": "Session language locked to: {lang}",
        "no_further_symptoms": "No further symptoms provided. Exiting diagnosis."

    },
    "ar": {
        "welcome": "👋 مرحبًا بك في نظام التشخيص الطبي بالذكاء الاصطناعي!",
        "input_method_prompt": "هل ترغب في إدخال الأعراض كنص أم كلام؟ (text/speech): ",
        "invalid_input_method": "إدخال غير صالح. يرجى كتابة \"text\" أو \"speech\".",
        "speech_file_prompt": "🎙 يرجى تقديم مسار ملفك الصوتي المسجل: ",
        "audio_transcription_error": "تعذر نسخ الصوت، أو كان الصوت فارغًا، أو حدث خطأ. يرجى المحاولة مرة أخرى أو التبديل إلى النص.",
        "no_audio_file": "لم يتم تقديم مسار ملف صوتي.",
        "text_symptoms_prompt": "🩺 يرجى وصف الأعراض الخاصة بك: ",
        "symptoms_required": "\nالأعراض مطلوبة للمتابعة. يرجى تقديم الأعراض الخاصة بك.",
        "must_provide_symptoms": "يجب عليك تقديم الأعراض للمتابعة.",
        "insufficient_symptoms_msg": "\n⚠️ الأعراض التي قدمتها ليست محددة بما يكفي للتشخيص.",
        "recommended_dept_insufficient": "🏥 القسم الموصى به: طبيب عام",
        "provide_more_details": "💬 يرجى تقديم مزيد من التفاصيل (مثل المدة، مكان الألم، الشدة).",
        "add_more_symptoms_prompt": "➕ أضف المزيد من الأعراض (أو اضغط على Enter للتوقف): ",
        "diagnosis_results_header": "\n✅ نتائج التشخيص:",
        "disease_label": "🩺 المرض: ",
        "confidence_label": " (الثقة: {confidence:.2f})",
        "recommended_dept_label": "   🏥 القسم الموصى به: ",
        "audio_file_not_found": "لم يتم العثور على الملف الصوتي في: {file_path}",
        "transcription_empty": "أسفر النسخ عن نص فارغ.",
        "transcription_error_detailed": "خطأ أثناء نسخ الصوت أو الترجمة: {e}",
        "transcribing_audio": "🎤 جاري نسخ الصوت...",
        "transcribed_text_label": "🗣 النص المنسوخ ({lang}): {text}",
        "translated_to_en_label": "🌍 تمت الترجمة إلى الإنجليزية للنموذج: {text}",
        "detected_text_lang_label": "تم اكتشاف لغة الإدخال (النص): {lang}",
        "lang_detection_error": "لم نتمكن من تحديد اللغة من النص، سنفترض الإنجليزية للنموذج. ستستخدم الواجهة اللغة الافتراضية.",
        "session_lang_set_debug": "تم تثبيت لغة الجلسة على: {lang}",
        "no_further_symptoms": "لم يتم تقديم المزيد من الأعراض. إنهاء التشخيص."
    }
}

In [43]:
def get_ui_text(key, **kwargs):
    global SESSION_OUTPUT_LANGUAGE
    #return ui_texts.get(SESSION_OUTPUT_LANGUAGE, ui_texts["en"]).get(key, key).format(**kwargs)

    template = ui_texts.get(SESSION_OUTPUT_LANGUAGE, ui_texts["en"]).get(key, key)
    try:
      return template.format(**kwargs)
    except (IndexError, KeyError):
      return template

In [32]:
def set_session_language_if_not_locked(detected_language_code):
    global SESSION_OUTPUT_LANGUAGE, SESSION_LANGUAGE_LOCKED
    if not SESSION_LANGUAGE_LOCKED:
        if detected_language_code == "ar":
            SESSION_OUTPUT_LANGUAGE = "ar"
        else:
            SESSION_OUTPUT_LANGUAGE = "en"
        SESSION_LANGUAGE_LOCKED = True
        # print(get_ui_text("session_lang_set_debug", lang=SESSION_OUTPUT_LANGUAGE)) # For debugging

In [92]:
def interactive_diagnosis():
    global SESSION_OUTPUT_LANGUAGE, SESSION_LANGUAGE_LOCKED
    SESSION_LANGUAGE_LOCKED = False
    SESSION_OUTPUT_LANGUAGE = "en"

    print(get_ui_text("welcome"))

    input_method = ""
    while input_method not in ["text", "speech"]:
        input_method = input(get_ui_text("input_method_prompt")).strip().lower()
        if input_method not in ["text", "speech"]:
            print(get_ui_text("invalid_input_method"))

    accumulated_symptoms_for_model = ""
    initial_symptoms_obtained = False

    while not initial_symptoms_obtained:
        current_symptoms_for_model_segment = ""
        detected_lang_for_segment = "en"

        if input_method == "speech":
            file_path = input(get_ui_text("speech_file_prompt")).strip()
            if file_path:
                current_symptoms_for_model_segment, detected_lang_for_segment = get_symptoms_from_audio(file_path)
                if not current_symptoms_for_model_segment:
                    print(get_ui_text("audio_transcription_error"))
            else:
                print(get_ui_text("no_audio_file"))
        elif input_method == "text":
            raw_symptoms = input(get_ui_text("text_symptoms_prompt")).strip()
            if raw_symptoms:
                try:
                    detected_lang_for_segment = detect(raw_symptoms)
                    print(get_ui_text("detected_text_lang_label" , lang=detected_lang_for_segment))
                    if detected_lang_for_segment == "ar":
                        output=translate_arabic_to_medical_english(raw_symptoms)
                        current_symptoms_for_model_segment = output
                        print(get_ui_text("translated_to_en_label", text=current_symptoms_for_model_segment))
                    else:
                        current_symptoms_for_model_segment = raw_symptoms
                except LangDetectException:
                    print(get_ui_text("lang_detection_error"))
                    detected_lang_for_segment = "en" # Default to English for UI if detection fails
                    current_symptoms_for_model_segment = raw_symptoms
            else:
                 print(get_ui_text("symptoms_required")) # if no text symptoms entered

        if current_symptoms_for_model_segment:
            set_session_language_if_not_locked(detected_lang_for_segment)
            accumulated_symptoms_for_model += " "+current_symptoms_for_model_segment
            initial_symptoms_obtained = True
        else:
            print(get_ui_text("must_provide_symptoms"))

    #diagnosis loop - accumulates symptoms
    while True:
        if not accumulated_symptoms_for_model.strip():

            print(get_ui_text("must_provide_symptoms"))
            break

        predictions_en = predict_disease(accumulated_symptoms_for_model)

        if predictions_en[0][0] == "Insufficient Symptoms":
            print(get_ui_text("insufficient_symptoms_msg"))
            print(get_ui_text("recommended_dept_insufficient"))
            print(get_ui_text("provide_more_details"))
            more_info_raw = input(get_ui_text("add_more_symptoms_prompt")).strip()

            if not more_info_raw:
                print(get_ui_text("no_further_symptoms"))
                 # Instead of breaking, get the current best predictions even if confidence is low
                predictions_en = predict_disease(accumulated_symptoms_for_model, threshold=0)
                print(get_ui_text("diagnosis_results_header"))
                for i, (disease_en, confidence, department_en) in enumerate(predictions_en):
                    disease_display = disease_en
                    department_display = department_en
                    if SESSION_OUTPUT_LANGUAGE == "ar":
                        if disease_en in df['Disease'].values:
                            disease_display=df[df['Disease']==disease_en]['arabic_disease'].values[0]
                        else:
                             disease_display =  "not found"
                        #disease_display = translate_with_model(disease_en, "ar", "en")
                        department_display = department_translations.get(department_en, department_en)

                    print(f"{i+1}. {get_ui_text('disease_label')}{disease_display}{get_ui_text('confidence_label', confidence=confidence)}")
                    print(f"   {get_ui_text('recommended_dept_label')}{department_display}\n")
                break

            more_symptoms_en = more_info_raw
            try:

                lang_of_more_info = detect(more_info_raw)
                if lang_of_more_info == "ar":
                    output=translate_arabic_to_medical_english(more_info_raw)
                    more_symptoms_en = output
                    print(get_ui_text("translated_to_en_label", text=current_symptoms_for_model_segment))
                    # Optional: print translation confirmation for additional symptoms
                    print(f"(Additional symptoms translated to EN: {more_symptoms_en})")
            except LangDetectException:
                print(get_ui_text("lang_detection_error"))
                detected_lang_for_segment = "en"


            accumulated_symptoms_for_model += " " + more_symptoms_en.strip()
            print(f"(Updated symptoms for model: {accumulated_symptoms_for_model})")
        else:
            print(get_ui_text("diagnosis_results_header"))
            for i, (disease_en, confidence, department_en) in enumerate(predictions_en):
                disease_display = disease_en
                department_display = department_en

                if SESSION_OUTPUT_LANGUAGE == "ar":
                    if disease_en in df['Disease'].values:
                        disease_display=df[df['Disease']==disease_en]['arabic_disease'].values[0]
                    else:
                        disease_display =  "not found"
                    #disease_display = translate_with_model(disease_en, "ar", "en")
                    department_display = department_translations.get(department_en, department_en)


                print(f"{i+1}. {get_ui_text('disease_label')}{disease_display}{get_ui_text('confidence_label', confidence=confidence)}")
                print(f"   {get_ui_text('recommended_dept_label')}{department_display}\n")

            break


In [79]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: زورى وجعانى وعندى احتقان
Detected input language (text): ar
🌍 Translated to English for model: Sore throat, pharyngeal congestion

✅ نتائج التشخيص:
1. 🩺 المرض: القلاع الفموي (عدوى الخميرة) (الثقة: 0.61)
      🏥 القسم الموصى به: طب الاسنان 

2. 🩺 المرض: التهاب الجيوب الأنفية الحاد (الثقة: 0.08)
      🏥 القسم الموصى به: انف واذن وحنجره

3. 🩺 المرض: خراج الأنف (الثقة: 0.04)
      🏥 القسم الموصى به: انف واذن وحنجره



In [96]:
if __name__ == "__main__":

     interactive_diagnosis()


👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: Dull, aching pain in the front of the knee, often behind or around the kneecap,Pain that worsens when going up or down stairs
Detected input language (text): en

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): Tenderness over the kneecap
(Updated symptoms for model:  Dull, aching pain in the front of the knee, often behind or around the kneecap,Pain that worsens when going up or down stairs Tenderness over the kneecap)

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): Feeling of the knee "giving w

In [80]:
if __name__ == "__main__":

     interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: My throat is sore and I have trouble swallowing
Detected input language (text): en

✅ Diagnosis Results:
1. 🩺 Disease: peritonsillar abscess (confidence:0.36)
      🏥 Recommended Department: ENT

2. 🩺 Disease: mucositis (confidence:0.19)
      🏥 Recommended Department: Dentistry

3. 🩺 Disease: abscess of the pharynx (confidence:0.14)
      🏥 Recommended Department: ENT



In [95]:
if __name__ == "__main__":

    interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: صداع وضيق في التنفس و غثيان و قيء
Detected input language (text): ar
🌍 Translated to English for model: headache, shortness of breath, nausea, vomiting

✅ نتائج التشخيص:
1. 🩺 المرض: فرط بوتاسيوم الدم (الثقة: 0.48)
      🏥 القسم الموصى به: أمراض الكلى

2. 🩺 المرض: نقص بوتاسيوم الدم (الثقة: 0.08)
      🏥 القسم الموصى به: أمراض الكلى

3. 🩺 المرض: رد فعل المخدرات (الثقة: 0.07)
      🏥 القسم الموصى به: حساسيه



In [ ]:
## old excute from old code

In [99]:
interactive_diagnosis()


👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: Headache  • Dizziness  • Tiredness (Fatigue)  • Blurred vision  • Itchiness
Detected input language (text): en

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): Numbness, tingling, burning, or weakness in hands
(Updated symptoms for model:  Headache  • Dizziness  • Tiredness (Fatigue)  • Blurred vision  • Itchiness Numbness, tingling, burning, or weakness in hands)

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): Feeling full quickly after eating
(Updated symptoms for model:  Headache  • Dizziness

In [100]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: Euphoria or feeling "high"  • Drowsiness, nodding off ("on the nod")  • Slowed breathing (respiratory depression - dangerous)  • Small, constricted pupils ("pinpoint pupils")  • Confusion
Detected input language (text): en

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): Slurred speech  • Poor coordination  • Nausea, vomiting  • Constipation  • Itching
(Updated symptoms for model:  Euphoria or feeling "high"  • Drowsiness, nodding off ("on the nod")  • Slowed breathing (respiratory depression - dangerous)  • Small, constricted pupils ("pinpoint pupils")  • Confusion Slurred speech  • Poor coordination  • Nausea, vomiting  • Constipation  • Itching)

⚠️ Your symptoms ar

In [101]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: Itchiness, mostly after a warm bath or shower. Numbness, tingling, burning or weakness in the hands, feet, arms or legs. A feeling of fullness soon after eating. Bloating or pain in the left upper stomach area due to an enlarged spleen. Unusual bleeding, such as a nosebleed or bleeding gums. Painful swelling of one joint, often the big toe. Shortness of breath and trouble breathing when lying down. Bone pain.
Detected input language (text): en

✅ Diagnosis Results:
1. 🩺 Disease: peripheral arterial disease (confidence:0.64)
      🏥 Recommended Department: Vascular Surgery

2. 🩺 Disease: pancreatic cancer (confidence:0.03)
      🏥 Recommended Department: Oncology

3. 🩺 Disease: primary immunodeficiency (confidence:0.03)
      🏥 Recommended Department: Immunology



In [93]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): speech
🎙 Please provide the path to your recorded audio file: /content/13.56.59_50a5128c.waptt.opus
🎤 Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣 Transcribed Text (ar): أنا عندي كحة وسخونية شديدة مع رشح
🌍 Translated to English for model: cough, high fever, rhinorrhea

✅ نتائج التشخيص:
1. 🩺 المرض: متلازمة التسنين (الثقة: 0.91)
      🏥 القسم الموصى به: طب الأطفال

2. 🩺 المرض: طفح حفاضات (الثقة: 0.02)
      🏥 القسم الموصى به: امراض جلدية وتجميلية

3. 🩺 المرض: مرض خلايا الدم البيضاء (الثقة: 0.01)
      🏥 القسم الموصى به: أمراض الدم



In [94]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): speech
🎙 Please provide the path to your recorded audio file: /content/at 22.00.16_6577fe4a.waptt.opus
🎤 Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣 Transcribed Text (ar): مشرق في الشارع مش متركزة العرق بيصب منك مصدعة مغا وجعيها مهمدة عصبها مرخية مرخية ورجليتي بتوجعك مشر
🌍 Translated to English for model: Difficulty concentrating, profuse sweating, headache, abdominal cramps, fatigue, generalized weakness, leg pain

✅ نتائج التشخيص:
1. 🩺 المرض: متلازمة الساق المضطربة (الثقة: 0.72)
      🏥 القسم الموصى به: أمراض الأعصاب

2. 🩺 المرض: رد فعل الإجهاد الحاد (الثقة: 0.07)
      🏥 القسم الموصى به: الطب النفسي

3. 🩺 المرض: صداع التوتر (الثقة: 0.01)
      🏥 القسم الموصى به: أمراض الأعصاب



In [104]:
interactive_diagnosis()

👋 Welcome to AI Medical Diagnosis System!
Would you like to enter symptoms as text or speech? (text/speech): text
🩺 Please describe your symptoms: There's this area down there that just won't stop itching. It feels sore too. I looked, and the skin looks kind of thick and maybe a bit lighter in color than the rest. It's been going on for weeks
Detected input language (text): en

⚠️ Your symptoms are not specific enough for a diagnosis.
🏥 Recommended Department: General Practitioner
💬 Please provide more details (e.g., duration, pain location, severity).
➕ Add more symptoms (or press Enter to stop): 
No further symptoms provided. Exiting diagnosis.

✅ Diagnosis Results:
1. 🩺 Disease: acne (confidence:0.11)
      🏥 Recommended Department: Dermatology

2. 🩺 Disease: seborrheic dermatitis (confidence:0.06)
      🏥 Recommended Department: Dermatology

3. 🩺 Disease: onychomycosis (confidence:0.05)
      🏥 Recommended Department: Dermatology

